In [ ]:
main_folder = "~/Documents/"

### HCP fODF Generation from dMRI Data

In [ ]:
# hcp fodf generation from dmri data
import os
import numpy as np
from utils.utils import HCP_LIST

order = 4

dwi_folder = f"{main_folder}hcp_large/normal_dwi/"
dwi_subfolder = "T1w"
fodf_folder = f"{main_folder}hcp_large/normal_dwi_fodf/"
fodf_subfolder = f"o{order}"

shell_file = f"{main_folder}shell_commands/fodfs_from_dmri_o{order}.sh"

f = open(shell_file, "w")

for hcp_number in HCP_LIST:
    hcp_folder = os.path.join(dwi_folder, hcp_number, dwi_subfolder)
    hcp_fodf_folder = os.path.join(fodf_folder, hcp_number, fodf_subfolder)
    f.write(f"mkdir -p {hcp_fodf_folder}\n")
    f.write(f"data2fodf -i {hcp_folder} -o {hcp_fodf_folder} -f\n")
    f.write("sleep 2m\n")

# delete unused files
fodf_folder = f"{main_folder}hcp_large/normal_dwi_fodf/"
fodf_subfolder = f"o{order}/Diffusion"

for hcp_number in HCP_LIST:
    hcp_fodf_folder = os.path.join(fodf_folder, hcp_number, fodf_subfolder)
    f.write(f"rm -r {hcp_fodf_folder}\n")

f.write("echo Done!\n")
f.close()

### Watson Signal Generation from fODF Data

In [ ]:
# watson signal generation from fodf data
import os
import numpy as np
from utils.utils import HCP_LIST

order = 4

fodf_folder = f"{main_folder}hcp_large/normal_dwi_fodf/"
fodf_subfolder = f"o{order}/mtdeconv/ankele/"

watson_folder = f"{main_folder}hcp_large/normal_dwi_watson/"
watson_subfolder = f"o{order}"
watson_backup_filename = "watson_backup.npz"
watson_tracking_filename = "watson_tracking_data.nrrd"
watson_peak_tracking_filename = "peak_tracking_data.nrrd"
watson_vvi_filename = "watson_vvi_cone_data.nrrd"

shell_file = f"{main_folder}shell_commands/watson_o{order}_from_fodf.sh"

f = open(shell_file, "w")

for hcp_number in HCP_LIST:
    hcp_fodf_folder = os.path.join(fodf_folder, hcp_number, fodf_subfolder)
    hcp_watson_folder = os.path.join(watson_folder, hcp_number, watson_subfolder)
    f.write(f"python {main_folder}dMRI_Watson_Fitting/watson-fitting --init given --initfile {main_folder}hcp_large/normal_dwi_lowrank/{hcp_number}/o4/fodf_peaks.nrrd --i {hcp_fodf_folder} -ob {os.path.join(hcp_watson_folder, watson_backup_filename)} -o {os.path.join(hcp_watson_folder, watson_tracking_filename)} -op {os.path.join(hcp_watson_folder, watson_peak_tracking_filename)} -vvi {os.path.join(hcp_watson_folder, watson_vvi_filename)}\n")
    f.write("sleep 2m\n")

f.write("echo Done!\n")
f.close()

### LowRank Signal Generation from fODF Data

In [10]:
import os
import numpy as np
from utils.utils import HCP_LIST

order = 4

fodf_folder = f"{main_folder}hcp_large/normal_dwi_fodf/"
fodf_subfolder = "mtdeconv/ankele/"

lowrank_folder = f"{main_folder}hcp_large/normal_dwi_lowrank/"
lowrank_subfolder = f"o{order}"

shell_file = f"{main_folder}shell_commands/lowrank_o{order}_from_fodf.sh"

f = open(shell_file, "w")

for hcp_number in HCP_LIST:
    lowrank_full_folder = os.path.join(lowrank_folder, hcp_number, lowrank_subfolder)
    f.write(f"mkdir -p {lowrank_full_folder}\n")

for hcp_number in HCP_LIST:
    fodf_full_folder = os.path.join(fodf_folder, hcp_number, fodf_subfolder)
    lowrank_full_folder = os.path.join(lowrank_folder, hcp_number, lowrank_subfolder)
    f.write(f"mkdir -p {lowrank_full_folder}\n")
    f.write(f"low-rank-k-approx {os.path.join(fodf_full_folder,'fodf.nrrd')} {os.path.join(lowrank_full_folder, 'fodf_peaks.nrrd')} &\n")

f.write("wait\n")
f.write("echo Done!\n")
f.close()

### fodf-sh Generation for Watson Tracking with fODF Interpolation

In [ ]:
import os
import numpy as np
from utils.utils import HCP_LIST

order = 4

dwi_folder = f"{main_folder}hcp_large/normal_dwi/"
dwi_subfolder = "T1w/Diffusion"
fodf_folder = f"{main_folder}hcp_large/normal_dwi_fodf/"
fodf_subfolder = "mtdeconv/ankele/"
fodf_sh_subfolder = f"sh_fodf/o{order}/"

shell_file = f"{main_folder}shell_commands/fodf_sh_o{order}_from_fodf.sh"

f = open(shell_file, "w")

for hcp_number in HCP_LIST:
    fodf_sh_full_folder = os.path.join(fodf_folder, hcp_number, fodf_sh_subfolder)
    f.write(f"mkdir -p {fodf_sh_full_folder}\n")

for hcp_number in HCP_LIST:
    dwi_full_folder = os.path.join(dwi_folder, hcp_number, dwi_subfolder)
    fodf_full_folder = os.path.join(fodf_folder, hcp_number, fodf_subfolder)
    fodf_sh_full_folder = os.path.join(fodf_folder, hcp_number, fodf_sh_subfolder)
    f.write(f"python {main_folder}dMRI_Watson_Fitting/watson-fodf-sh-generation --i {fodf_full_folder} -m {os.path.join(dwi_full_folder, 'data.nii.gz')} -o {os.path.join(fodf_sh_full_folder, 'fodf.nrrd')} \n")

f.write("echo Done!\n")
f.close()

### Tracking from Watson or LowRank data

In [579]:
# watson prob tracking from watson data
import os
import numpy as np
from utils.utils import Tract, Method, HCP_LIST

methods = [Method.Watson.value]
orders = (np.ones(len(methods)) * 4).astype(int)
tract = Tract.CST_left.value
number_of_seeds = 20000
append = ""#"_s"
in_parallel = True
is_ply = False
wmmin = 0.4

def generate_tracking_files(methods, orders, tract, number_of_seeds, append, wmmin, is_ply, in_parallel, hcp_list):
    shell_files = []
    for i, method in enumerate(methods):
        order = orders[i]
        fodf_folder = f"{main_folder}hcp_large/normal_dwi_fodf/"
        fodf_subfolder = "mtdeconv/ankele/"
        fodf_sh_subfolder = f"sh_fodf/o{order}/"

        watson_folder = f"{main_folder}hcp_large/normal_dwi_watson/"
        watson_subfolder = f"o{order}"
        watson_tracking_filename = "watson_tracking_data.nrrd"
        watson_peak_tracking_filename = "peak_tracking_data.nrrd"

        lowrank_folder = f"{main_folder}hcp_large/normal_dwi_lowrank/"
        lowrank_subfolder = f"o{order}"
        lowrank_tracking_filename = "fodf_peaks.nrrd"

        watson_tracking_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
        watson_tracking_subfolder = f"o{order}"
        watson_tracking_seedsubfolder = "seeding"

        registration_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
        registration_subfolder = "registration/"

        shell_file = f"{main_folder}shell_commands/{tract}_{method}{append}_{number_of_seeds}_o{order}_tracking.sh"
        
        shell_files.append(shell_file)
        f = open(shell_file, "w")

        for hcp_number in hcp_list:
            fodf_full_folder = os.path.join(fodf_folder, hcp_number, fodf_subfolder)
            watson_full_folder = os.path.join(watson_folder, hcp_number, watson_subfolder)
            lowrank_full_folder = os.path.join(lowrank_folder, hcp_number, lowrank_subfolder)
            watson_tracking_full_folder = os.path.join(watson_tracking_folder, hcp_number, watson_tracking_subfolder, tract)
            watson_seeding_full_folder = os.path.join(watson_tracking_folder, hcp_number, watson_tracking_seedsubfolder, tract)
            registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder)
            watson_tracking_output_filename = f"{method}_{number_of_seeds}{append}.{'ply' if is_ply else 'tck'}"
            additional_flags = ""
            if method == Method.Watson.value or method == Method.WatsonDProb.value:
                infile = os.path.join(watson_full_folder,watson_tracking_filename)
                prob = "Watson"
                interpolation = "FACT"
                if method == Method.WatsonDProb.value:
                    additional_flags = "--watsonprobdirselection"
            elif method == Method.WatsonIntp.value or method == Method.WatsonIntpDProb.value:
                infile = os.path.join(watson_full_folder,watson_tracking_filename)
                prob = "Watson"
                interpolation = "TrilinearFODFWatson"
                # wmvolume as additional parameter
                additional_flags = f"--wmvolume {os.path.join(fodf_full_folder, 'wmvolume.nrrd')}"
                if method == Method.WatsonIntpDProb.value:
                    additional_flags += " --watsonprobdirselection"
                # use different folder for fodf with sh fodf
                fodf_full_folder = fodf_full_folder = os.path.join(fodf_folder, hcp_number, fodf_sh_subfolder)
            elif method == Method.WatsonPeakProb.value:
                infile = os.path.join(watson_full_folder,watson_peak_tracking_filename)
                prob = "ScalarNew"
                interpolation = "FACT"
            elif method == Method.WatsonPeakDet.value:
                infile = os.path.join(watson_full_folder,watson_peak_tracking_filename)
                prob = "Deterministic"
                interpolation = "FACT"
            elif method == Method.LowRank.value:
                infile = os.path.join(lowrank_full_folder,lowrank_tracking_filename)
                prob = "ScalarNew"
                interpolation = "FACT"
            elif method == Method.LowRankIntp.value:
                infile = os.path.join(lowrank_full_folder,lowrank_tracking_filename)
                prob = "ScalarNew"
                interpolation = "TrilinearFODF"
            elif method == Method.LowRankDet.value:
                infile = os.path.join(lowrank_full_folder,lowrank_tracking_filename)
                prob = "Deterministic"
                interpolation = "FACT"

            f.write(f"mkdir -p {watson_tracking_full_folder}\n")
            f.write(f"prob-tracking --i {fodf_full_folder} --infile {infile} {additional_flags} --seedpoints {os.path.join(watson_seeding_full_folder,'seeds')}_{number_of_seeds}.pts -o {os.path.join(watson_tracking_full_folder,watson_tracking_output_filename)} --dist 0 --interpolation {interpolation} --prob {prob} --wmmin {wmmin} --rank 3 --var 6 --exp 3 --features seedpoint angle prob_chosen {'&' if in_parallel else ''}\n")
            #f.write("sleep 2m\n")
        if in_parallel:
            f.write("wait\n")
        f.write("echo Done!\n")
        f.close()

    return shell_files

### TRK to TCK

In [ ]:
# trk to tck
import os
import numpy as np
from utils.utils import Tract, HCP_LIST

tract = f"{Tract.CST_left.value}.trk"

hcp_trk_folder = f"{main_folder}hcp_large/HCP105_GT/"
hcp_trk_subfolder = "tracts/"

shell_file = f"{main_folder}shell_commands/trk_to_tck_{tract}.sh"

f = open(shell_file, "w")

for hcp_number in HCP_LIST:
    hcp_full_trk_folder = os.path.join(hcp_trk_folder, hcp_number, hcp_trk_subfolder)
    f.write(f"python {main_folder}dMRI_Watson_Fitting/helpers/trk2tck.py {os.path.join(hcp_full_trk_folder, tract)}\n")
    #f.write("sleep 2m\n")

f.write("echo Done!\n")
f.close()

### Create Registration .mat for all Subjects

In [97]:
import os
import numpy as np
from utils.utils import Tract, HCP_LIST

tract = f"{Tract.CST_left.value}.trk"

reference_folder = f"{main_folder}hcp_large/normal_dwi/792766/T1w/"

subject_folder = f"{main_folder}hcp_large/normal_dwi/"
subject_subfolder = "T1w/"

registration_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
registration_subfolder = "registration/"

registration_data_file = "T1w_acpc_dc_restore_1.25.nii.gz"

shell_file = f"{main_folder}shell_commands/subjects_registration.sh"

f = open(shell_file, "w")

for hcp_number in HCP_LIST:
    subject_full_folder = os.path.join(subject_folder, hcp_number, subject_subfolder)
    registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder)
    f.write(f"mkdir -p {registration_full_folder}\n")
    f.write(f"flirt -in {os.path.join(reference_folder, registration_data_file)} -ref {os.path.join(subject_full_folder, registration_data_file)} -omat {os.path.join(registration_full_folder, 'ref2img.mat')}\n")
    #f.write("sleep 2m\n")

f.write("echo Done!\n")
f.close()

### Creation of Seedpoints

In [ ]:
import numpy as np
from utils.utils import Tract, HCP_LIST

tracts = [Tract.CC.value]

number_of_seeds = 25000
seed_points_file = "seedmask.pts"

initial_direction = True
direction = np.array([1,0,0], dtype=float)
direction /= np.linalg.norm(direction)
dir_str = f"{direction[0]},{direction[1]},{direction[2]}"

for tract in tracts:
    ref_registration_folder = os.path.join(f"{main_folder}hcp_large/792766_registration/", tract)

    subject_folder = f"{main_folder}hcp_large/normal_dwi/"
    subject_subfolder = "T1w/"

    gt_tracts_folder = f"{main_folder}hcp_large/HCP105_GT/"
    gt_tracts_subfolder = "tracts/"

    fodf_folder = f"{main_folder}hcp_large/normal_dwi_fodf/"
    fodf_subfolder = "mtdeconv/ankele/"

    registration_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
    registration_subfolder = "registration/"

    seeding_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
    seeding_subfolder = "seeding/"

    registration_data_file = "T1w_acpc_dc_restore_1.25.nii.gz"

    def pts_to_niigz(folder, pts_file, out_file):
        return f"python {main_folder}dMRI_Watson_Fitting/helpers/pts2nrrd.py -i {os.path.join(folder,pts_file)} -o {os.path.join(folder,out_file)}.nii.gz -r -n {main_folder}hcp_large/normal_dwi_fodf/792766/mtdeconv/ankele/wmvolume.nrrd\n"

    def transform_mask(ref_registration_folder, mask_name, subject_full_folder, registration_data_file, registration_full_folder, seeding_full_folder, tract):
        return f"flirt -in {os.path.join(ref_registration_folder, mask_name)}.nii.gz -ref {os.path.join(subject_full_folder, registration_data_file)} -out {os.path.join(seeding_full_folder,tract,mask_name)}.nii.gz -applyxfm -init {os.path.join(registration_full_folder, 'ref2img.mat')}"

    shell_file = f"{main_folder}shell_commands/{tract}_seeding_{number_of_seeds}.sh"

    f = open(shell_file, "w")

    # create nii.gz for seed mask
    f.write(pts_to_niigz(ref_registration_folder, seed_points_file, seed_points_file[:-4]))

    # register seedmask to subjects
    for hcp_number in HCP_LIST:
        seeding_full_folder = os.path.join(seeding_folder, hcp_number, seeding_subfolder)
        f.write(f"mkdir -p {os.path.join(seeding_full_folder,tract)}\n")

    for hcp_number in HCP_LIST:
        subject_full_folder = os.path.join(subject_folder, hcp_number, subject_subfolder)
        registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder)
        seeding_full_folder = os.path.join(seeding_folder, hcp_number, seeding_subfolder)
        f.write(f"{transform_mask(ref_registration_folder, seed_points_file[:-4], subject_full_folder, registration_data_file, registration_full_folder, seeding_full_folder, tract)} &\n")

    f.write("wait\n")

    for hcp_number in HCP_LIST:
        registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder)
        seeding_full_folder = os.path.join(seeding_folder, hcp_number, seeding_subfolder)
        gt_tracts_full_folder = os.path.join(gt_tracts_folder, hcp_number, gt_tracts_subfolder)
        fodf_full_folder = os.path.join(fodf_folder, hcp_number, fodf_subfolder)
        f.write(f"python {main_folder}dMRI_Watson_Fitting/helpers/create_seeds.py -maskfile {os.path.join(seeding_full_folder,tract,seed_points_file[:-4])}.nii.gz -n {number_of_seeds} -o {os.path.join(seeding_full_folder,tract,'seeds')}_{number_of_seeds}.pts -affine {os.path.join(fodf_full_folder, 'wmvolume.nrrd')} -st {os.path.join(gt_tracts_full_folder, tract)}.trk {f'-direction {dir_str}' if initial_direction else ''} &\n")

    f.write("wait\n")
    f.write("echo Done!\n")
    f.close()

### Inclusion- and Exclusion regions from given Points and Mapping to all Subjects

In [785]:
# Strings für Dateinamen, für zwei Regionen oder mehr Regionen, wo nur eine berührt werden muss können Subarrays erstellt werden
from utils.utils import Tract, HCP_LIST

ref_registration_folder = os.path.join(f"{main_folder}hcp_large/792766_registration/", tract)

subject_folder = f"{main_folder}hcp_large/normal_dwi/"
subject_subfolder = "T1w/"

registration_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
registration_subfolder = "registration/"

registration_data_file = "T1w_acpc_dc_restore_1.25.nii.gz"

def pts_to_niigz(folder, pts_file, out_file):
    return f"python {main_folder}dMRI_Watson_Fitting/helpers/pts2nrrd.py -i {os.path.join(folder,pts_file)} -o {os.path.join(folder,out_file)}.nii.gz -r -n {main_folder}hcp_large/normal_dwi_fodf/792766/mtdeconv/ankele/wmvolume.nrrd"

def transform_mask(ref_registration_folder, mask_name, subject_full_folder, registration_data_file, registration_full_folder, tract):
    return f"flirt -in {os.path.join(ref_registration_folder, mask_name)}.nii.gz -ref {os.path.join(subject_full_folder, registration_data_file)} -out {os.path.join(registration_full_folder,tract,mask_name)}.nii.gz -applyxfm -init {os.path.join(registration_full_folder, 'ref2img.mat')}"

shell_file = f"{main_folder}shell_commands/{tract}_create_filtering_masks.sh"

f = open(shell_file, "w")

# create nii.gz for all inclusion and exclusion regions
for i, inc in enumerate(inclusion):
    # multiple where lines must cross any of them
    if type(inc) == list:
        fslmath_string = "fslmaths "
        for inc_m in inc:
            f.write(f"{pts_to_niigz(ref_registration_folder, inc_m, inc_m[:-4])} &\n")
            fslmath_string += os.path.join(ref_registration_folder, f"{inc_m[:-4]}.nii.gz") + " -add "
        fslmath_string = fslmath_string[:-5] + os.path.join(ref_registration_folder, f"{i}_inc.nii.gz")
        f.write("wait\n")
        f.write(f"{fslmath_string} &\n")
    else:
        f.write(f"{pts_to_niigz(ref_registration_folder, inc, f'{i}_inc')} &\n")

# create nii.gz for exclusion regions
for i, exc in enumerate(exclusion):
    f.write(f"{pts_to_niigz(ref_registration_folder, exc, f'{i}_exc')} &\n")

# create nii.gz for inclusion truncation masks
for i, inc in enumerate(mask_in):
    # multiple where lines must cross any of them
    if type(inc) == list:
        fslmath_string = "fslmaths "
        for inc_m in inc:
            f.write(f"{pts_to_niigz(ref_registration_folder, inc_m, inc_m[:-4])} &\n")
            fslmath_string += os.path.join(ref_registration_folder, f"{inc_m[:-4]}.nii.gz") + " -add "
        fslmath_string = fslmath_string[:-5] + os.path.join(ref_registration_folder, f"{i}_mask_in.nii.gz")
        f.write("wait\n")
        f.write(f"{fslmath_string} &\n")
    else:
        f.write(f"{pts_to_niigz(ref_registration_folder, inc, f'{i}_mask_in')} &\n")

# create nii.gz for exclusion truncation masks
for i, exc in enumerate(mask_ex):
    f.write(f"{pts_to_niigz(ref_registration_folder, exc, f'{i}_mask_ex')} &\n")

f.write("wait\n")

for hcp_number in HCP_LIST:
    registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder)
    f.write(f"rm -r {os.path.join(registration_full_folder,tract)}\n")
    f.write(f"mkdir {os.path.join(registration_full_folder,tract)}\n")

# register regions to subjects
for hcp_number in HCP_LIST:
    subject_full_folder = os.path.join(subject_folder, hcp_number, subject_subfolder)
    registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder)
    for i in range(len(inclusion)):
        f.write(f"{transform_mask(ref_registration_folder, f'{i}_inc', subject_full_folder, registration_data_file, registration_full_folder, tract)} &\n")
    for i in range(len(exclusion)):
        f.write(f"{transform_mask(ref_registration_folder, f'{i}_exc', subject_full_folder, registration_data_file, registration_full_folder, tract)} &\n")
    for i in range(len(mask_in)):
        f.write(f"{transform_mask(ref_registration_folder, f'{i}_mask_in', subject_full_folder, registration_data_file, registration_full_folder, tract)} &\n")
    for i in range(len(mask_ex)):
        f.write(f"{transform_mask(ref_registration_folder, f'{i}_mask_ex', subject_full_folder, registration_data_file, registration_full_folder, tract)} &\n")

f.write("wait\n")
f.write("echo Done!\n")
f.close()

f = open(shell_file, "r")
print(shell_file)
#print(f.read())

/home/jonah/MEGA/shell_commands/CG_left_create_filtering_masks.sh


### Filterung: Trakt mit erstellten Regionen filtern für alle Patienten

In [ ]:
import os
from utils.utils import Tract, Method, HCP_LIST

methods = [Method.Watson.value]
orders = (np.ones(len(methods)) * 4).astype(int)
tract = Tract.ILF_left.value
append = ""#"_s"
number_of_seeds = 5000
is_ply = False
in_parallel = True
cci = True
cci_val = 3

def generate_filtering_files(methods, orders, tract, number_of_seeds, append, is_ply, cci, cci_val, in_parallel):
    shell_files = []
    for i, method in enumerate(methods):
        order = orders[i]
        filename = method
        watson_tracking_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
        watson_tracking_subfolder = f"o{order}"

        registration_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
        registration_subfolder = "registration/"

        gt_tracts_folder = f"{main_folder}hcp_large/HCP105_GT/"
        gt_tracts_subfolder = "tracts/"

        shell_file = f"{main_folder}shell_commands/{tract}_{method}{append}_{number_of_seeds}_o{order}_filter_tractograms.sh"

        shell_files.append(shell_file)

        f = open(shell_file, "w")

        if is_ply:
            for hcp_number in HCP_LIST:
                watson_tracking_full_folder = os.path.join(watson_tracking_folder, hcp_number, watson_tracking_subfolder, tract)
                watson_tractogram_filename = f"{filename}_{number_of_seeds}{append}"
                f.write(f"python {main_folder}dMRI_Watson_Fitting/helpers/ply2tck.py -i {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}.ply -o {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}.tck {'&' if in_parallel else ''}\n")

        if in_parallel:
            f.write("wait\n")

        inc_ex_trunc_mask = False

        # inclusion truncation mask
        for hcp_number in HCP_LIST:
            watson_tracking_full_folder = os.path.join(watson_tracking_folder, hcp_number, watson_tracking_subfolder, tract)
            registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder, tract)
            watson_tractogram_filename = f"{filename}_{number_of_seeds}{append}"

            tckedit_string = f"tckedit {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}.tck {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}_filtered.tck "

            mask_in_files = [x for x in os.listdir(registration_full_folder) if '_mask_in' in x] # streamlines exiting the mask will be truncated

            if len(mask_in_files) != 0:
                inc_ex_trunc_mask = True
                for mask_in_file in mask_in_files:
                    tckedit_string += f"-mask {os.path.join(registration_full_folder,mask_in_file)} "

                f.write(f"{tckedit_string} -force {'&' if in_parallel else ''}\n")

        if in_parallel:
            f.write("wait\n")

        # exclusion truncation mask
        for hcp_number in HCP_LIST:
            watson_tracking_full_folder = os.path.join(watson_tracking_folder, hcp_number, watson_tracking_subfolder, tract)
            registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder, tract)
            watson_tractogram_filename = f"{filename}_{number_of_seeds}{append}"

            tckedit_string = f"tckedit {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}{'_filtered' if inc_ex_trunc_mask else ''}.tck {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}_filtered.tck "

            mask_ex_files = [x for x in os.listdir(registration_full_folder) if '_mask_ex' in x] # streamlines entering the mask will be truncated

            if len(mask_ex_files) != 0:
                inc_ex_trunc_mask = True
                for mask_ex_file in mask_ex_files:
                    tckedit_string += f"-mask {os.path.join(registration_full_folder,mask_ex_file)} "

                f.write(f"{tckedit_string} -inverse -force {'&' if in_parallel else ''}\n")

        if in_parallel:
            f.write("wait\n")

        # inclusion and exclusion masks
        for hcp_number in HCP_LIST:
            watson_tracking_full_folder = os.path.join(watson_tracking_folder, hcp_number, watson_tracking_subfolder, tract)
            registration_full_folder = os.path.join(registration_folder, hcp_number, registration_subfolder, tract)
            watson_tractogram_filename = f"{filename}_{number_of_seeds}{append}"

            tckedit_string = f"tckedit {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}{'_filtered' if inc_ex_trunc_mask else ''}.tck {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}_filtered.tck "

            include_files = [x for x in os.listdir(registration_full_folder) if '_inc' in x]
            exclude_files = [x for x in os.listdir(registration_full_folder) if '_exc' in x]
            
            if len(include_files) + len(exclude_files) != 0:
                for inc_file in include_files:
                    tckedit_string += f"-include {os.path.join(registration_full_folder,inc_file)} "
                for exc_file in exclude_files:
                    tckedit_string += f"-exclude {os.path.join(registration_full_folder,exc_file)} "
    
                f.write(f"{tckedit_string} -force {'&' if in_parallel else ''}\n")

        if in_parallel:
            f.write("wait\n")

        if cci:
            for hcp_number in HCP_LIST:
                watson_tracking_full_folder = os.path.join(watson_tracking_folder, hcp_number, watson_tracking_subfolder, tract)
                watson_tractogram_filename = f"{filename}_{number_of_seeds}{append}"
                gt_tracts_full_folder = os.path.join(gt_tracts_folder, hcp_number, gt_tracts_subfolder)
                f.write(f"python {main_folder}dMRI_Watson_Fitting/helpers/filter_by_cci.py -st {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}_filtered.tck -rst {os.path.join(gt_tracts_full_folder,tract)}.trk -o {os.path.join(watson_tracking_full_folder, watson_tractogram_filename)}_filtered.tck -cci {cci_val} {'&' if in_parallel else ''}\n")

        if in_parallel:
            f.write("wait\n")

        f.write("echo Done!\n")
        f.close()
    
    return shell_files

generate_filtering_files(methods, orders, tract, number_of_seeds, append, is_ply, cci, cci_val, in_parallel)

### Compute Dice Score, OL and OR

In [443]:
from utils.utils import Method, Tract, HCP_LIST

methods = [Method.Watson.value]
orders = (np.ones(len(methods)) * 4).astype(int)
tract = Tract.CST_left.value
append = ""
number_of_seeds = 20000
smoothing = 1
dice_threshold = 1

# WICHTIG: in_parallel nicht auf Laptop, sonst frozen
in_parallel = True

def generate_dice_files(methods, orders, tract, number_of_seeds, append, smoothing, in_parallel, dice_threshold = 1):
    shell_files = []
    for i, method in enumerate(methods):
        order = orders[i]

        gt_tracts_folder = f"{main_folder}hcp_large/HCP105_GT/"
        gt_tracts_subfolder = "tracts/"
        
        watson_tracking_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"
        watson_tracking_subfolder = f"o{order}"
        
        shell_file = f"{main_folder}shell_commands/{tract}_{method}{append}_{number_of_seeds}_o{order}_dice.sh"

        shell_files.append(shell_file)
        
        f = open(shell_file, "w")
        
        for i, hcp_number in enumerate(HCP_LIST):
            watson_tracking_full_folder = os.path.join(watson_tracking_folder, hcp_number, watson_tracking_subfolder, tract)
            gt_tracts_full_folder = os.path.join(gt_tracts_folder, hcp_number, gt_tracts_subfolder)
            f.write(f"python {main_folder}dMRI_Watson_Fitting/compute_dice.py -st {os.path.join(watson_tracking_full_folder,method)}_{number_of_seeds}{append}_filtered.tck -rst {os.path.join(gt_tracts_full_folder,tract)}.trk -save {os.path.join(watson_tracking_full_folder, 'dice')}_{method}_{number_of_seeds}{append}.npy -dt {dice_threshold} -smoothing {smoothing} {'&' if in_parallel else ''}\n")
            if in_parallel and i % 8 == 7:
                f.write("wait\n")
        
        if in_parallel:
            f.write("wait\n")
        f.write("echo Done!\n")
        f.close()

    return shell_files

### Read Dice Data

In [ ]:
import numpy as np
from utils.utils import Method, Tract, HCP_LIST
import os

import matplotlib
from matplotlib import pyplot as plt

tracts = [Tract.CST_left.value]
methods = [Method.WatsonIntp.value, Method.LowRankIntp.value, Method.Watson.value, Method.LowRank.value]
appends = ['','','','']
orders = [4,4,4,4]
labels = ['Iterative Watson model', 'Iterative Low-rank model', 'Watson model', 'Low-rank model']
number_of_seeds_list = (np.ones((len(methods),len(tracts)))).astype(int)
dice_names_set = ['Dice', 'Overreach', 'Overlap', 'Seeds after Filtering']
markers = ['o', 'X', "^","v","P","s"]

watson_tracking_folder = f"{main_folder}hcp_large/normal_dwi_watson_tracking/"

dice_set = np.zeros((len(tracts), len(methods), len(HCP_LIST), 4))
for h, tract in enumerate(tracts):
    for i, method in enumerate(methods):
        watson_tracking_subfolder = f"o{orders[i]}"
        for j, hcp_number in enumerate(HCP_LIST):
            dice_file = f"{os.path.join(watson_tracking_folder, hcp_number, watson_tracking_subfolder, tract, 'dice')}_{method}_{number_of_seeds_list[i,h]}{appends[i]}.npy"
            dice_data = np.load(dice_file)
            dice_set[h,i,j,:len(dice_data)] = dice_data

means = dice_set.mean(axis=2)
std_devs = dice_set.std(axis=2)

# plot dice means
for d, dice_name in enumerate(dice_names_set):
    plt.figure(figsize=(6, 12))
    for i in range(len(methods)):
        plt.plot(means[:,i,d], range(0,len(means[:,i,d])), markers[i], label=labels[i])

    plt.xlabel(dice_name)
    plt.ylabel('Tract')
    plt.yticks(range(0,len(means[:,i,d])), [tract.replace('_',' ') for tract in tracts])
    plt.legend()
    plt.show()

#### p-Values

In [ ]:
from scipy import stats

watsonintp_all_dice = dice_set[:,0,:,0].reshape(-1)
lowrankintp_all_dice = dice_set[:,1,:,0].reshape(-1)
watson_all_dice = dice_set[:,2,:,0].reshape(-1)
lowrank_all_dice = dice_set[:,3,:,0].reshape(-1)

#perform Friedman Test
friedman_result = stats.friedmanchisquare(watsonintp_all_dice, lowrankintp_all_dice, watson_all_dice, lowrank_all_dice)
print(friedman_result)

import scikit_posthocs as sp
import numpy as np

#combine three groups into one array
data = np.array([watsonintp_all_dice, lowrankintp_all_dice, watson_all_dice, lowrank_all_dice])

#perform Nemenyi post-hoc test
nemenyi_table = sp.posthoc_nemenyi_friedman(data.T)
nemenyi_table

### Combined Tracking, Filtering & Dice

In [85]:
methods = [Method.Watson.value, Method.LowRank.value, Method.WatsonIntp.value, Method.LowRankIntp.value]
orders = (np.ones(len(methods)) * 4).astype(int)
tracts = [Tract.CG_left.value]
append = "" # arbitrary filename additions
number_of_seeds_list = (np.ones(len(tracts)) * 5000).astype(int)

In [ ]:
from utils.utils import Method, Tract
import numpy as np

# tracking params
wmmin = 0.4
in_parallel_tracking = True
is_ply = False

# filtering params
cci = True
cci_val = 2
in_parallel_filtering = True

# dice params
smoothing = 2
dice_threshold = 1
in_parallel_dice = True

# generating sh files
s_shell_files = []
for j, tract in enumerate(tracts):
    tracking_shell_files = generate_tracking_files(methods, orders, tract, number_of_seeds_list[j], append, wmmin, is_ply, in_parallel_tracking, HCP_LIST[:1])
    filtering_shell_files = generate_filtering_files(methods, orders, tract, number_of_seeds_list[j], append, is_ply, cci, cci_val, in_parallel_filtering)
    dice_shell_files = generate_dice_files(methods, orders, tract, number_of_seeds_list[j], append, smoothing, in_parallel_dice, dice_threshold)

    shell_file = f"{main_folder}shell_commands/combined_pipeline_{tract}_{number_of_seeds_list[j]}{append}.sh"
    s_shell_files.append(shell_file)
    
    f = open(shell_file, "w")
    for i in range(len(methods)):
        f.write(f"{tracking_shell_files[i]}\n")
        f.write(f"{filtering_shell_files[i]}\n")
        f.write(f"{dice_shell_files[i]}\n")

    f.write("echo All Done!\n")
    f.close()

print("chmod +x *.sh")
for shell_file in s_shell_files:
    print(shell_file)
print("echo All All Done!\n\n")